pip install docplex gurobipy

2. Preparación del modelo base
El siguiente es un ejemplo genérico para un modelo F-MDRP basado en las restricciones SCF (Single Commodity Flow):

In [ ]:
from docplex.mp.model import Model
import gurobipy as gp
from gurobipy import GRB
import numpy as np

import numpy as np

def read_instance(file_path):
    """
    Lee una instancia de archivo .dat y devuelve los parámetros del problema.
    Formato esperado:
    - Primera línea: Número de clientes
    - Segunda línea: Número de depósitos disponibles
    - Siguientes líneas: Coordenadas de los depósitos (x, y)
    - Luego: Coordenadas de los clientes (x, y)
    - Capacidades de los vehículos
    - Capacidades de los depósitos
    - Demandas de los clientes
    - Costos de apertura de los depósitos
    - Costos de apertura de una ruta (costo de un vehículo)
    - Indicación de si los costos son enteros (0) o reales (1)
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Leer el número de clientes y depósitos
    num_customers = int(lines[0].strip())
    num_depots = int(lines[1].strip())

    # Leer las coordenadas de los depósitos
    depots_coords = []
    line_index = 2
    for i in range(num_depots):
        line = lines[line_index].strip()
        # Asegurarnos de que la línea no esté vacía y manejar la tabulación
        if line:
            x, y = map(float, line.replace('\t', ' ').split())
            depots_coords.append((x, y))
        line_index += 1

    # Leer las coordenadas de los clientes
    customers_coords = []
    for i in range(num_customers):
        line = lines[line_index].strip()
        # Asegurarnos de que la línea no esté vacía y manejar la tabulación
        if line:
            x, y = map(float, line.replace('\t', ' ').split())
            customers_coords.append((x, y))
        line_index += 1

    # Leer la capacidad de los vehículos
    vehicle_capacity = float(lines[line_index].strip())
    line_index += 1

    # Leer las capacidades de los depósitos
    depots_capacities = list(map(float, lines[line_index].strip().replace('\t', ' ').split()))
    line_index += 1

    # Leer las demandas de los clientes
    customers_demands = list(map(float, lines[line_index].strip().replace('\t', ' ').split()))
    line_index += 1

    # Leer los costos de apertura de los depósitos
    opening_costs = list(map(float, lines[line_index].strip().replace('\t', ' ').split()))
    line_index += 1

    # Leer el costo de apertura de una ruta (costo de un vehículo)
    vehicle_route_cost = float(lines[line_index].strip())
    line_index += 1

    # Leer si los costos son enteros (0) o reales (1)
    cost_type = int(lines[line_index].strip())

    # Generar la matriz de costos entre depósitos y clientes (calculada usando la distancia euclidiana)
    depot_customer_costs = np.zeros((num_depots, num_customers))
    for i in range(num_depots):
        for j in range(num_customers):
            depot_x, depot_y = depots_coords[i]
            customer_x, customer_y = customers_coords[j]
            depot_customer_costs[i, j] = np.sqrt((depot_x - customer_x)**2 + (depot_y - customer_y)**2)

    # Generar la matriz de costos entre los depósitos (sin conexión entre depósitos)
    depot_depot_costs = np.zeros((num_depots, num_depots))
    for i in range(num_depots):
        for j in range(i + 1, num_depots):
            depot_x1, depot_y1 = depots_coords[i]
            depot_x2, depot_y2 = depots_coords[j]
            depot_depot_costs[i, j] = depot_depot_costs[j, i] = np.sqrt((depot_x1 - depot_x2)**2 + (depot_y1 - depot_y2)**2)

    # Empaquetar todos los parámetros en un diccionario
    data = {
        'num_customers': num_customers,
        'num_depots': num_depots,
        'depots_coords': depots_coords,
        'customers_coords': customers_coords,
        'vehicle_capacity': vehicle_capacity,
        'depots_capacities': depots_capacities,
        'customers_demands': customers_demands,
        'opening_costs': opening_costs,
        'vehicle_route_cost': vehicle_route_cost,
        'cost_type': cost_type,
        'depot_customer_costs': depot_customer_costs,
        'depot_depot_costs': depot_depot_costs
    }

    return data








# Implementación del modelo con CPLEX
def setup_model_scf_cplex(depots, clients, costs, vehicles_per_depot):
    mdl = Model(name="F-MDRP SCF")

    # Variables de decisión
    x = mdl.binary_var_dict(
        [(i, j) for i in depots + clients for j in depots + clients if i != j],
        name="x"
    )
    f = mdl.continuous_var_dict(
        [(i, j) for i in depots + clients for j in clients if i != j],
        name="f"
    )

    # Función objetivo: Minimizar los costos de ruta
    mdl.minimize(mdl.sum(costs[i][j] * x[i, j] for i in depots + clients for j in depots + clients if i != j))

    # Restricciones
    # 1. Cada cliente es visitado exactamente una vez
    for c in clients:
        mdl.add_constraint(mdl.sum(x[c, j] for j in depots + clients if c != j) == 1)
        mdl.add_constraint(mdl.sum(x[j, c] for j in depots + clients if c != j) == 1)

    # 2. Restricciones de flujo en los depósitos
    for d in depots:
        mdl.add_constraint(mdl.sum(x[d, j] for j in clients) <= vehicles_per_depot[d])
        mdl.add_constraint(mdl.sum(x[j, d] for j in clients) <= vehicles_per_depot[d])

    # 3. Eliminación de subrutas (SCF)
    for c in clients:
        mdl.add_constraint(mdl.sum(f[i, c] for i in depots + clients if i != c) -
                           mdl.sum(f[c, j] for j in clients if c != j) == 1)

        for i, j in [(i, j) for i in depots + clients for j in clients if i != j]:
            mdl.add_constraint(f[i, j] <= len(clients) * x[i, j])

    return mdl


# Implementación del modelo con Gurobi
def solve_with_gurobi(depots, clients, costs, vehicles_per_depot):
    mdl = gp.Model("F-MDRP SCF")

    # Variables de decisión
    x = mdl.addVars([(i, j) for i in depots + clients for j in depots + clients if i != j],
                    vtype=GRB.BINARY, name="x")
    f = mdl.addVars([(i, j) for i in depots + clients for j in clients if i != j],
                    vtype=GRB.CONTINUOUS, name="f")

    # Función objetivo
    mdl.setObjective(
        gp.quicksum(costs[i][j] * x[i, j] for i in depots + clients for j in depots + clients if i != j),
        GRB.MINIMIZE
    )

    # Restricciones
    # 1. Cada cliente es visitado exactamente una vez
    for c in clients:
        mdl.addConstr(gp.quicksum(x[c, j] for j in depots + clients if c != j) == 1)
        mdl.addConstr(gp.quicksum(x[j, c] for j in depots + clients if c != j) == 1)

    # 2. Restricciones de flujo en los depósitos
    for d in depots:
        mdl.addConstr(gp.quicksum(x[d, j] for j in clients) <= vehicles_per_depot[d])
        mdl.addConstr(gp.quicksum(x[j, d] for j in clients) <= vehicles_per_depot[d])

    # 3. Eliminación de subrutas (SCF)
    for c in clients:
        mdl.addConstr(gp.quicksum(f[i, c] for i in depots + clients if i != c) -
                      gp.quicksum(f[c, j] for j in clients if c != j) == 1)

        for i, j in [(i, j) for i in depots + clients for j in clients if i != j]:
            mdl.addConstr(f[i, j] <= len(clients) * x[i, j])

    # Solución
    mdl.setParam("TimeLimit", 3600)  # Límite de tiempo de 1 hora
    mdl.optimize()

    return mdl


# Función principal
def main():
    # Ruta de la instancia .dat
    instance_path = "/home/gaspar/Comparacion_cplex_gurobi/Gurobi_Cplex_comparation/instances/coord20-5-1.dat"  # Cambiar por la ruta de tu archivo .dat

    # Leer la instancia
    data = read_instance(instance_path)

    # Datos extraídos
    depots = list(range(data['num_depots']))
    clients = list(range(data['num_depots'], data['num_depots'] + data['num_customers']))
    costs = data['depot_customer_costs']
    vehicles_per_depot = {d: 3 for d in depots}  # Número arbitrario de vehículos por depósito

    # Resolver con CPLEX
    cplex_model = setup_model_scf_cplex(depots, clients, costs, vehicles_per_depot)
    cplex_model.parameters.timelimit = 3600  # Límite de tiempo de 1 hora
    cplex_model.solve()

    print("Resultados CPLEX:")
    print(cplex_model.objective_value)

    # Resolver con Gurobi
    gurobi_model = solve_with_gurobi(depots, clients, costs, vehicles_per_depot)

    print("Resultados Gurobi:")
    if gurobi_model.status == GRB.OPTIMAL:
        print(gurobi_model.objVal)


if __name__ == "__main__":
    main()



ValueError: could not convert string to float: '38\t5'